In [12]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
import optuna
import warnings
from tqdm import tqdm
warnings.simplefilter('ignore')

In [13]:
# 設定値
class CFG:
    # 変更するパラメータ
    n_folds = 5 # 公差検証の分割数(多くて20)
    n_trials = 100 # ハイパーパラメータチューニングの試行回数(100)
    device_type = "cpu"
    # device_type = "gpu"
    boosting_type = "gbdt"
    # boosting_type = "dart"
    
    
    # その他設定値
    learning_rate = 0.01
    seed = 3407 
    target_col = 'Class'
    num_boost_round = 50500
    early_stopping_round = 300
    verbose_eval = 0  # この数字を1にすると学習時のスコア推移がコマンドライン表示される

In [14]:
# BASE_DIR = '/kaggle/input/icr-identify-age-related-conditions/'
BASE_DIR = '../data'
train_df = pd.read_csv(f'{BASE_DIR}/train.csv')
greeks_df = pd.read_csv(f'{BASE_DIR}/greeks.csv')
test_df = pd.read_csv(f'{BASE_DIR}/test.csv')
submission_df = pd.read_csv(f'{BASE_DIR}/sample_submission.csv')

test_df[CFG.target_col] = -1
all_df = pd.concat([train_df, test_df])

BC, CLはいらんかも

In [15]:
numerical_features = ['AB', 'AF', 'AH', 'AM', 'AR', 'AX', 'AY', 'AZ', 'BC', 'BD ', 'BN',
       'BP', 'BQ', 'BR', 'BZ', 'CB', 'CC', 'CD ', 'CF', 'CH', 'CL', 'CR', 'CS',
       'CU', 'CW ', 'DA', 'DE', 'DF', 'DH', 'DI', 'DL', 'DN', 'DU', 'DV', 'DY',
       'EB', 'EE', 'EG', 'EH', 'EL', 'EP', 'EU', 'FC', 'FD ', 'FE', 'FI',
       'FL', 'FR', 'FS', 'GB', 'GE', 'GF', 'GH', 'GI', 'GL']
categorical_features = ['EJ']
features = numerical_features + categorical_features

### balanced loglossの計算（学習で使う？）

In [16]:
# 前処理
def Preprocessing(input_df: pd.DataFrame)->pd.DataFrame:
    output_df = input_df.copy()
    output_df['EJ'] = input_df['EJ'].replace({'A': 0, 'B': 1})
    return output_df

all_df = Preprocessing(all_df)

train_df = all_df[all_df[CFG.target_col] != -1].copy()
test_df = all_df[all_df[CFG.target_col] == -1].copy()

In [17]:
# 評価基準
def balanced_log_loss(y_true, y_pred):
    N = len(y_true)

    # Nc is the number of observations
    N_1 = np.sum(y_true == 1, axis=0)
    N_0 = np.sum(y_true == 0, axis=0)

    # In order to avoid the extremes of the log function, each predicted probability 𝑝 is replaced with max(min(𝑝,1−10−15),10−15)
    y_pred = np.maximum(np.minimum(y_pred, 1 - 1e-15), 1e-15)

    # balanced logarithmic loss
    loss_numerator = - (1/N_0) * np.sum((1 - y_true) * np.log(1-y_pred)) - (1/N_1) * np.sum(y_true * np.log(y_pred))

    return loss_numerator / 2

# Classの０，１の割合をそれぞれ計算
def calc_log_loss_weight(y_true):
    nc = np.bincount(y_true)
    w0, w1 = 1/(nc[0]/y_true.shape[0]), 1/(nc[1]/y_true.shape[0])
    return w0, w1

In [18]:
def objective(trial):
    # light-gbm設定値
    lgb_params = {
        # 探索するパラメータ
        'verbosity': -1, # 学習途中の情報を表示するかどうか
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        
        # 固定値
        "boosting_type": CFG.boosting_type,
        "device_type": CFG.device_type,
        "objective": "binary",
        "learning_rate": CFG.learning_rate,
        "metric": "binary_logloss",
        'seed': CFG.seed,
        'n_jobs': -1, # -1でコア数をマックスで使う
        'is_unbalance':True, # 不均衡データの場合にTrueにする
    }
    
    scores = []
    # K-分割交差検証(層化抽出法)
    kfold = StratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
    pbar = tqdm(enumerate(kfold.split(train_df, train_df[CFG.target_col])))
    for fold, (train_index, valid_index) in pbar:
    # for fold, (train_index, valid_index) in enumerate(kfold.split(train_df, train_df[CFG.target_col])):
        # 現在の試行回数を出力
        pbar.set_description("[train] trials {}".format(trial.number+1))
        
        x_train = train_df[features].iloc[train_index]
        y_train = train_df[CFG.target_col].iloc[train_index]
        x_valid = train_df[features].iloc[valid_index]
        y_valid = train_df[CFG.target_col].iloc[valid_index]
        
        # 訓練データの重みを計算
        train_w0, train_w1 = calc_log_loss_weight(y_train)
        # 検証データの重みを計算
        valid_w0, valid_w1 = calc_log_loss_weight(y_valid)
        # 訓練データをlgb用に変換
        lgb_train = lgb.Dataset(x_train, y_train, weight=y_train.map({0: train_w0, 1: train_w1}), categorical_feature=categorical_features)
        # lgb_train = lgb.Dataset(x_train, y_train, categorical_feature=categorical_features)
        # 検証データをlgb用に変換
        lgb_valid = lgb.Dataset(x_valid, y_valid, weight=y_valid.map({0: valid_w0, 1: valid_w1}), categorical_feature=categorical_features)
        # lgb_valid = lgb.Dataset(x_valid, y_valid, categorical_feature=categorical_features)
        
        model = lgb.train(
                    params = lgb_params,
                    train_set = lgb_train,
                    num_boost_round = CFG.num_boost_round,
                    valid_sets = [lgb_train, lgb_valid],
                    early_stopping_rounds = CFG.early_stopping_round,
                    verbose_eval = CFG.verbose_eval,
                    # 学習段階でbalanced_log_lossを使う場合はコメントアウト外す
                    # feval = lgb_metric,
                )
        # 予測
        preds = model.predict(x_valid)
        # 予測値をラベルに変換
        # pred_labels = np.rint(preds)
        # 評価
        # val_score = balanced_log_loss(y_valid, pred_labels)
        val_score = balanced_log_loss(y_valid, preds)
        
        scores.append(val_score)
    # クロスバリデーションの平均値を計算
    mean_score = np.mean(scores)
    
    return mean_score

In [19]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=CFG.n_trials)

[I 2023-06-03 16:37:00,260] A new study created in memory with name: no-name-ad4d7ee9-b9dc-47e5-9854-1becff7422ed
[train] trials 1: : 5it [00:07,  1.43s/it]
[I 2023-06-03 16:37:07,403] Trial 0 finished with value: 0.32094636606285587 and parameters: {'lambda_l1': 0.00015551180232562748, 'lambda_l2': 0.00018159023485480913, 'num_leaves': 136, 'feature_fraction': 0.8573122025422735, 'bagging_fraction': 0.34142413338657995, 'min_child_samples': 9}. Best is trial 0 with value: 0.32094636606285587.
[train] trials 2: : 5it [00:07,  1.53s/it]
[I 2023-06-03 16:37:15,040] Trial 1 finished with value: 0.30759567415171085 and parameters: {'lambda_l1': 4.7676669032534814e-08, 'lambda_l2': 1.4014796780414881e-08, 'num_leaves': 177, 'feature_fraction': 0.9978515353572892, 'bagging_fraction': 0.5046112909546834, 'min_child_samples': 17}. Best is trial 1 with value: 0.30759567415171085.
[train] trials 3: : 5it [00:03,  1.28it/s]
[I 2023-06-03 16:37:18,950] Trial 2 finished with value: 0.25394575364690

In [23]:
best_params = study.best_trial.params
best_value = study.best_value
print("best_param: {}\n\nbest_value: {}".format(best_params, best_value))

best_param: {'lambda_l1': 3.9679492838164295e-07, 'lambda_l2': 6.757138144718837e-07, 'num_leaves': 7, 'feature_fraction': 0.6627683984494798, 'bagging_fraction': 0.9135956967880636, 'min_child_samples': 32}

best_value: 0.2123926514129733


In [21]:
"""
# best_paramsには固定値が保存されないので改めて設定
best_params["boosting_type"] = CFG.boosting_type
best_params["device_type"] = CFG.device_type
best_params["seed"] = CFG.seed
best_params["n_jobs"] = -1
best_params["is_unbalance"] = True
best_params["objective"] = "binary"
best_params["learning_rate"] = 0.005
best_params["metric"] = "binary_logloss"
best_params["verbosity"] = -1

# 設定したハイパーパラメータを基にモデルを作成
model = lgb.LGBMClassifier(**best_params)
# 学習
model.fit(train_df[features], train_df[CFG.target_col])
"""

'\n# best_paramsには固定値が保存されないので改めて設定\nbest_params["boosting_type"] = CFG.boosting_type\nbest_params["device_type"] = CFG.device_type\nbest_params["seed"] = CFG.seed\nbest_params["n_jobs"] = -1\nbest_params["is_unbalance"] = True\nbest_params["objective"] = "binary"\nbest_params["learning_rate"] = 0.005\nbest_params["metric"] = "binary_logloss"\nbest_params["verbosity"] = -1\n\n# 設定したハイパーパラメータを基にモデルを作成\nmodel = lgb.LGBMClassifier(**best_params)\n# 学習\nmodel.fit(train_df[features], train_df[CFG.target_col])\n'

In [22]:
"""
# 予測
prediction = model.predict_proba(test_df.drop(["Id", "Class"], axis=1))
# 提出用に値を変換
prediction = max(min(prediction, 1-10**(-15)), 10**(-15))
submission = pd.DataFrame(columns = submission_df.columns)
submission['Id'] = test_df['Id']
submission[['class_0','class_1']] = prediction
submission.to_csv('submission.csv',index=False)
submission
"""

'\n# 予測\nprediction = model.predict_proba(test_df.drop(["Id", "Class"], axis=1))\n# 提出用に値を変換\nprediction = max(min(prediction, 1-10**(-15)), 10**(-15))\nsubmission = pd.DataFrame(columns = submission_df.columns)\nsubmission[\'Id\'] = test_df[\'Id\']\nsubmission[[\'class_0\',\'class_1\']] = prediction\nsubmission.to_csv(\'submission.csv\',index=False)\nsubmission\n'